NOTE
--

This is a CNN version instead of PFF; for individual study; will be merged later.

Shu Kong 

20190130

Predictive Filter Flow for Non-Uniform Motion Blur Removal
================
**Author**: `Shu Kong <https://www.ics.uci.edu/~skong2/pff.html>`

**Date**: Nov. 2018

This is a jupyter demo for showing how to train, evaluate, and visualize the model of non-uniform motion deblur. It squeezes the following parts. Please read through, uncomment lines and run accordingly for your interest.

-  defining model architecture
-  defining the loss function
-  training protocal
-  evaluation protocal
-  analysis using PCA, k-means and t-SNE.


Others to notice: 
- Pytorch is used for this work.
- Please manually install packages as shown below if necessary
- For the training set, the DIV2K dataset is not uploaded due to its large size. If you want to train the model using the whole training set, please download manually, chop each image into overlapping 512x512 sub-images, and put them in the corresponding folder (read through this jupyter script to see where to put them:-) For demonstration and turtorial purpose, other small-scale training images are uploaded here.

import packages
------------------

Some packages are installed automatically if you use Anaconda. As pytorch is used here, you are expected to install that in your machine. 

In [ ]:
from __future__ import print_function, division
import os, random, time, copy
from skimage import io, transform
import numpy as np
import os.path as path
import scipy.io as sio
from scipy import misc
from scipy import ndimage, signal
import scipy
import pickle

import math
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
from skimage import data, img_as_float
from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_psnr as psnr

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler 
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets, models, transforms

from libs_deblur.utils.metrics import *
from libs_deblur.utils.flow_functions import *
from libs_deblur.models.pixel_embedding_model import *
from libs_deblur.datasetMotionBlur import *
from libs_deblur.trainvalGaussBlur_cnn import *
import libs_deblur.pyblur
import warnings # ignore warnings
warnings.filterwarnings("ignore")

 Setup config parameters
 -----------------
 
 There are several things to setup, like which GPU to use, where to read images and save files, etc. Please read and understand this. By default, you should be able to run this script smoothly by changing nothing.

In [ ]:
# set device, which gpu to use.
device ='cpu'
if torch.cuda.is_available(): device='cuda:0'

################## set attributes for this project/experiment ##################
# config result folder
exp_dir = './libs_deblur' # experiment directory, used for reading the init model
project_name = 'demo' # name this project as "demo", used for saving files
path_to_root = './libs_deblur/dataset' # where to fetch data


batch_size = 16 # small batch size for demonstration; using larger batch size (like 56 and 64) for training

embedding_dim = 16 # dimension of the learned embedding space
kernel_size = 17 # the kernel size in the filter flow
cropSize = [64, 64] # patch size for training the model
sigmaMin=0.5
sigmaMax=2

lambda_norm = 0.1
total_epoch_num = 500 # total number of epoch in training
base_lr = 0.0005 # base learning rate

torch.cuda.device_count()
torch.cuda.empty_cache()

save_dir = os.path.join(exp_dir, project_name) # where to save the log file and trained models.
print(save_dir)    
if not os.path.exists(save_dir): os.makedirs(save_dir)
log_filename = os.path.join(save_dir, 'train.log')

Define model architecture
---------

Here is the definition of the model architecture. 

In [ ]:
class SiamesePixelEmbed(nn.Module):
    def __init__(self, emb_dimension=64, filterSize=11, device='cpu', pretrained=False):
        super(SiamesePixelEmbed, self).__init__()
        self.device = device
        self.emb_dimension = emb_dimension  
        self.PEMbase = PixelEmbedModelResNet18(emb_dimension=self.emb_dimension, pretrained=pretrained)  
        self.rawEmbFeature1 = 0
        self.rawEmbFeature2 = 0        
        self.embFeature1_to_2 = 0
        self.embFeature1_to_2 = 0
        self.filterSize = filterSize
        self.filterSize2Channel = self.filterSize**2
                
        self.ordered_embedding = nn.Sequential(            
            nn.Conv2d(self.emb_dimension, self.filterSize2Channel, kernel_size=3, padding=1, bias=False),
            nn.ReLU(True),
            nn.BatchNorm2d(self.filterSize2Channel),     
            nn.Conv2d(self.filterSize2Channel, self.filterSize2Channel, kernel_size=3, padding=1, bias=False),
            nn.ReLU(True),
            nn.BatchNorm2d(self.filterSize2Channel),            
            nn.Conv2d(self.filterSize2Channel, 1, kernel_size=3, padding=1, bias=True)
        )
        
        
    def forward(self, inputs1):        
        self.rawEmbFeature1 = self.PEMbase.forward(inputs1)        
        self.embFeature1_to_2 = self.ordered_embedding(self.rawEmbFeature1)        
        #self.embFeature1_to_2 = F.softmax(self.embFeature1_to_2, 1)        
        return self.embFeature1_to_2

Define loss function
---------

Here is the L1 loss at pixel level, including how to warp the image with the predictive filter flow.

In [ ]:
class LossOrderedPairReconstruction(nn.Module):
    def __init__(self, device='cpu'):
        super(LossOrderedPairReconstruction, self).__init__()
        self.device = device
        self.reconstructImage = 0
        
    def forward(self, image1, image2):
        N,C,H,W = image1.size()
        self.reconstructImage = image1
        diff = self.reconstructImage - image2               
        diff = torch.abs(diff)       
        totloss = torch.sum(torch.sum(torch.sum(torch.sum(diff))))        
        return totloss/(N*C*H*W)    

Setup dataset
-----------

Here is where to fetch training and testing dataset.

In [ ]:
################## dataset ###################
transform4Image = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((127.,127.,127.),(127.,127.,127.)) # (mean, std)
    ]) # (mean, std)

whole_datasets = {set_name: 
                  Dataset4MotionBlur(root_dir=path_to_root,
                                     size=cropSize, set_name=set_name, 
                                     transform=transform4Image, 
                                     sigmaMin=sigmaMin, sigmaMax=sigmaMax)
                  for set_name in ['train', 'val']}


dataloaders = {set_name: DataLoader(whole_datasets[set_name], 
                                    batch_size=batch_size,
                                    shuffle=set_name=='train', 
                                    num_workers=5) # num_work can be set to batch_size
               for set_name in ['train', 'val']}

dataset_sizes = {set_name: len(whole_datasets[set_name]) for set_name in ['train', 'val']}
print(dataset_sizes)

Visualizing some training images
-----------

Visualizing some training image pairs, as well as the random blurring kernel.

In [ ]:
sample = iter(dataloaders['train']).next()
kernelList, imgList1, imgList2 = sample

imgList1 = imgList1.to(device)
imgList2 = imgList2.to(device)


figWinNumHeight, figWinNumWidth, subwinCount = batch_size/4, 4*3, 1
plt.figure(figsize=(15,8), dpi=88, facecolor='w', edgecolor='k') # figsize -- inch-by-inch
plt.clf()

for sampleIndex in range(batch_size):
    # visualize blurry image
    plt.subplot(figWinNumHeight,figWinNumWidth,subwinCount)
    subwinCount += 1
    noisy_img = imgList1[sampleIndex].cpu().numpy().squeeze().transpose((1,2,0))
    noisy_img = (noisy_img+1)/2    
    noisy_img = noisy_img.clip(0,1)        
    plt.imshow(noisy_img, cmap='gray')    
    plt.axis('off')
    plt.title('blurry img')
    
    
    plt.subplot(figWinNumHeight,figWinNumWidth,subwinCount)
    subwinCount += 1
    img = imgList2[sampleIndex].cpu().numpy().squeeze().transpose((1,2,0))
    img = (img+1)/2
    img = img.clip(0,1)        
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.title('original img')
    
    plt.subplot(figWinNumHeight,figWinNumWidth,subwinCount)
    subwinCount += 1
    kernel = kernelList[sampleIndex].cpu().numpy().squeeze()

    plt.imshow(kernel, cmap='gray')
    plt.axis('off')
    plt.title('kernel')

Training demo
-------------

Run this cell to train the model (essentially resuming the training over the model provided here). Stop it if you do not want to keep training. This is just for demonstrating how to train and how to tweak the architecture.


In [ ]:
################## init model ###################
initModel = SiamesePixelEmbed(emb_dimension=embedding_dim, 
                              filterSize=kernel_size,
                              device=device, pretrained=False)

initModel.load_state_dict(torch.load(os.path.join(exp_dir,'epoch-445.paramOnly')))
initModel.to(device);


# decreasing the momentum in batch normalization, as the default is too large for this work.
allLayer_dict = initModel.state_dict()
child_counter = 0
for child in initModel.PEMbase.children():
    for i in range(len(child)):        
        if 'BatchNorm2d' in str(type(child[i])): 
            child[i].momentum=0.001
            #print(child[i])            

for i in range(len(initModel.ordered_embedding)):    
    if 'BatchNorm2d' in str(type(initModel.ordered_embedding[i])): 
            initModel.ordered_embedding[i].momentum=0.001
            #print(initModel.ordered_embedding[i])
            
child_counter = 0
for child in initModel.children():
    #print(" child", child_counter, "is:")
    #print(child)
    child_counter += 1            

In [ ]:
################## loss function ###################
loss_1_to_2 = LossOrderedPairReconstruction(device=device)

loss_l1norm = nn.L1Loss(size_average=True)

optimizer_ft = optim.Adam([{'params': initModel.PEMbase.parameters()},
                           {'params': initModel.ordered_embedding.parameters(), 'lr': base_lr},                           
                         ], lr=base_lr)


# Decay LR by a factor of 0.5 every int(total_epoch_num/5) epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=int(total_epoch_num/5), gamma=0.5)

################## start training ###################
fn = open(log_filename,'w')
fn.write(log_filename+'\t'+device+'\n\n')
#fn.write(path.basename(__file__)+'\n\n')
fn.close()
file_to_note_bestModel = os.path.join(save_dir,'note_bestModel.log')
fn = open(file_to_note_bestModel, 'w')
fn.write('Record of best models on the way.\n')
fn.close()

model_ft = train_model(initModel, dataloaders, dataset_sizes, 
                       loss_1_to_2, 
                       optimizer_ft, exp_lr_scheduler,
                       num_epochs=total_epoch_num, 
                       work_dir=save_dir, device=device)

Leaving blank
=====

This is the end of the demo. Should have questions, please contact through the following

### Shu Kong


### aimerykong - at - gmail dot com



If you find anything inspires you, please cite


    @inproceedings{kong2018PPF,
      title={Image Reconstruction with Predictive Filter Flow},
      author={Kong, Shu and Fowlkes, Charless},
      booktitle={arxiv},
      year={2018}
    }
